# DB2-Salesforce connector: Tools mapping to authors

In [ ]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'AuthorToolAssociation__c'

from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')


salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors
Obtained Salesforce access token ...... True


In [ ]:
import pandas as pd
import time
import datetime


## Obtain tool information from DB2 

In [ ]:
# connect with DB2

# tool_authors
sql_query = "select * from jos_tool_authors"

toolauthors_df = pd.read_sql_query(sql_query, nanohub_db)

# display
toolauthors_df.head(2)

,toolname,revision,uid,ordering,version_id,name,organization
5838,mosfetsat,66,29294,2,2755,Tanya Faltens,Purdue University
8210,mif,6,29294,2,3561,Tanya Faltens,Purdue University
8732,crystal_viewer,394,29294,4,3714,Tanya Faltens,Purdue University
8921,crystal_viewer,395,29294,4,3775,Tanya Faltens,Purdue University
9777,tinwhis,14,29294,2,4284,Tanya Faltens,
10899,engdata,4,29294,1,4954,Tanya Faltens,


In [ ]:
ta_df = toolauthors_df[['toolname', 'uid']].drop_duplicates()

,toolname,uid
5838,mosfetsat,29294
8210,mif,29294
8732,crystal_viewer,29294
9777,tinwhis,29294
10899,engdata,29294


## Obtain Salesforce IDs

In [5]:
# create DB2 to Salesforce API object

Obtained Salesforce access token ...... True


In [6]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_user_ID_df = db_s.query_data('SELECT Id, nanoHUB_user_ID__c FROM Contact where nanoHUB_user_ID__c != NULL')

sf_user_ID_df['nanoHUB_user_ID__c'] = sf_user_ID_df['nanoHUB_user_ID__c'].astype('int')

# display
sf_user_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000TmJFWAA3
{"id":"7505w00000TmJFWAA3","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T06:30:43.000+0000","systemModstamp":"2021-01-21T06:30:43.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmJFWAA3","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T06:30:43.000+0000","systemModstamp":"2021-01-21T06:30:43.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmJFWAA3","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T06:30:43.000+0000","systemModst

,Id,nanoHUB_user_ID__c
0,0035w000031Vsp1AAC,998
1,0035w000031Vsp2AAC,1683
2,0035w000031Vsp3AAC,1684


In [7]:
# get Salesforce ID for citations
sf_tool_df = db_s.query_data('SELECT Id, Tool_name__c FROM nanoHUB_tools__c')

# display
sf_tool_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000TmJJxAAN
{"id":"7505w00000TmJJxAAN","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T06:41:26.000+0000","systemModstamp":"2021-01-21T06:41:26.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmJJxAAN","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T06:41:26.000+0000","systemModstamp":"2021-01-21T06:41:26.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmJJxAAN","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T06:4

,Id,Tool_name__c
0,a0s5w00000k5MOwAAM,ellipsom
1,a0s5w00000k5MOxAAM,hydrolab
2,a0s5w00000k5MOyAAM,huckel


## Match data with Salesforce format

In [8]:
# make sure all strings are lower cases
sf_tool_df['Tool_name__c'] = sf_tool_df['Tool_name__c'].astype('str').apply(str.lower)

In [9]:
# merge SF citation and contact IDs into user-citation DF
ct_tolink_df = pd.merge(ta_df, sf_tool_df, how='inner', left_on='toolname', right_on='Tool_name__c')\
                           .rename(columns={'Id':'SF_ID_tool'})
                                            
ct_tolink_df = pd.merge(ct_tolink_df, sf_user_ID_df, how='inner', left_on='uid', right_on='nanoHUB_user_ID__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

# display
ct_tolink_df.head(2)

,toolname,uid,SF_ID_tool,Tool_name__c,SF_ID_contact,nanoHUB_user_ID__c
0,adept,4645,a0s5w00000k5MP7AAM,adept,0035w000034JHCCAA4,4645
1,adeptnpt,4645,a0s5w00000k5MWdAAM,adeptnpt,0035w000034JHCCAA4,4645


In [10]:
# create a new column for object name
ct_tolink_df['Name'] = ct_tolink_df.apply(lambda x: '%s_%s'%(x.nanoHUB_user_ID__c, x.Tool_name__c), axis=1)

In [11]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ct_tolink_df['Name']
df_sf['Contact__c'] = ct_tolink_df['SF_ID_contact']
df_sf['Tool__c']       = ct_tolink_df['SF_ID_tool']
sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,4645_adept,4645_adeptnpt
Contact__c,0035w000034JHCCAA4,0035w000034JHCCAA4
Tool__c,a0s5w00000k5MP7AAM,a0s5w00000k5MWdAAM


## To Salesforce Sales Cloud CRM

In [12]:
# create DB2 to Salesforce API object
db_s = salesforce
# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [13]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000TmJKHAA3
hello
[Success] CSV upload successful. Job ID = 7505w00000TmJKHAA3
[Success] Closing job successful. Job ID = 7505w00000TmJKHAA3


In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-21T06:42:23.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000TmJKHAA3',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'AuthorToolAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'UploadComplete',
 'systemModstamp': '2021-01-21T06:42:24.000+0000',
 'totalProcessingTime': 0}
